In [10]:
import os

In [11]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\Text-Summarizer-Project'

In [16]:
os.chdir("Text-Summarizer-Project")

In [17]:
%pwd

'c:\\Users\\abhay\\Desktop\\ML Project\\Text-Summarizer-Project'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_files: Path
    unzip_dir: Path

In [29]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
    
        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_files = config.local_data_files,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [31]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_files
            )
            logger.info(f"{filename} download! with following info \n{headers}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_files))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Functions return None 
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



                
        

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-06-24 10:00:23,588: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-24 10:00:23,592: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-24 10:00:23,594: INFO: common: Created file directory at: artifacts]
[2025-06-24 10:00:23,595: INFO: common: Created file directory at: artifacts/data_ingestion]
[2025-06-24 10:00:23,595: INFO: 570811703: File already exists of size: ~7718 KB]
